# Characterizing the vertical structure of oxygen in the ocean

Argovis provides an API that indexes and distributes numerous oceanographic datasets with detailed query parameters, enabling you to search and download only and exactly data of interest. 

This notebook guides users in exploring the vertical structure of oxygen in the ocean using ocean observations and gridded products (i.e. products that provide maps of oxygen in the ocean at different depths, starting from sparse point measurements). Please note that while the focus in the following is on oxygen, the same notebook can be used to visualize other fields (see possible input parameters for e.g. salinity included in commented lines in between lines starting with #+++).

## Learning goals

- Generate an average vertical profile of oxygen for a region in the North Atlantic and for a region in the North Pacific Ocean using Argo oxygen data.  
- Describe the features of the two profiles, noting similarities and differences.  
- Discuss the reasons for the differences between the two. 

## Setup: Register an API key

In order to allocate Argovis's limited computing resources fairly, users are encouraged to register and request a free API key. This works like a password that identifies your requests to Argovis. To do so:

 - Visit [https://argovis-keygen.colorado.edu/](https://argovis-keygen.colorado.edu/)
 - Fill out the form under _New Account Registration_
 - An API key will be emailed to you shortly.
 
Treat this API key like a password - don't share it or leave it anywhere public. If you ever forget it or accidentally reveal it to a third party, see the same website above to change or deactivate your token.

Put your API key in the quotes in the variable below before moving on (please note that if you proceed without including anything in the quotes, the notebook may still work, yet it may also give an error):

In [1]:
API_KEY=''

## Setup: import libraries and functions

In [2]:
# Let's start importing libraries and functions we will need
from argovisHelpers import helpers as avh

import matplotlib.colors as mcolors
import matplotlib.pyplot as plt # e.g. to change the size of figures created from functions
import copy

#import gsw
# import numpy as np
#import matplotlib.pyplot as plt

import sys
sys.path.append('./work_in_progress')
# let's import functions from the .py file in the current folder
from Argovis_tasks_helpers import get_route,list_values_for_parameter_to_api_query,show_variable_names_for_collections
from Argovis_tasks_helpers import get_api_output_formatted_list_1var_for_regions_and_timeranges, get_api_output_formatted_list_1var_for_parameter, api_output_formatted_list_1var_plot_lons_lats_map,api_output_formatted_list_1var_plot_profiles,api_output_formatted_list_1var_plot_map,api_output_formatted_list_1var_plot_horizontal_and_time_ave,api_output_formatted_list_parameter_2d_plot
from Argovis_tasks_helpers import api_output_formatted_list_include_gsw_fields

## Setup: define parameters to query data of interest

Let's start with listing available data collections in Argovis. Please find more information about each product at https://argovis.colorado.edu/about.

In [3]:
# API call showing collections available for each route, along with a short description

#### to complete


Let's indicate the collections of interest, i.e. what datasets in Argovis you would like to use as they include the oceanic property of interest. We will also list the oceanic properties available in each collection (so that we can use the variable name of interest in the next code cell below). **Please first run the code with the current settings to familiarize yourself with the notebook: after that, if needed, then change the settings and run the notebook again.**

In [4]:
# for a list of collections, please see the Argovis swagger page

#### in the following we set parameters to compare ocean profiles with a gridded product
selection_params = {}

#+++ example to use Argo and cchdo/easyocean profile data, as well as the glodap gridded product (which provides time mean fields)
selection_params['collections']  = ['argo', 'cchdo', 'easyocean', 'grids/glodap']

#+++ example to use Argo profile data and the glodap gridded product (which provides time mean fields)
# selection_params['collections']  = ['argo', 'grids/glodap']
#+++

#+++ example to use Argo profile data and the Roemmich and Gilson monthly gridded product
# selection_params['collections']  = ['argo', 'grids/rg09']
#+++


#### let's print to screen some info based on the selected collections (to be able to select further parameters in the next cell)
vars_lists = show_variable_names_for_collections(collections_list=selection_params['collections'],API_KEY=API_KEY,verbose=True)


https://argovis-api.colorado.edu/argo/vocabulary?parameter=data
>>>>> argo
['bbp470', 'bbp470_argoqc', 'bbp532', 'bbp532_argoqc', 'bbp700', 'bbp700_2', 'bbp700_2_argoqc', 'bbp700_argoqc', 'bisulfide', 'bisulfide_argoqc', 'cdom', 'cdom_argoqc', 'chla', 'chla_argoqc', 'chla_fluorescence', 'chla_fluorescence_argoqc', 'cndc', 'cndc_argoqc', 'cp660', 'cp660_argoqc', 'down_irradiance380', 'down_irradiance380_argoqc', 'down_irradiance412', 'down_irradiance412_argoqc', 'down_irradiance443', 'down_irradiance443_argoqc', 'down_irradiance490', 'down_irradiance490_argoqc', 'down_irradiance555', 'down_irradiance555_argoqc', 'down_irradiance665', 'down_irradiance665_argoqc', 'down_irradiance670', 'down_irradiance670_argoqc', 'downwelling_par', 'downwelling_par_argoqc', 'doxy', 'doxy2', 'doxy2_argoqc', 'doxy3', 'doxy3_argoqc', 'doxy_argoqc', 'nitrate', 'nitrate_argoqc', 'ph_in_situ_total', 'ph_in_situ_total_argoqc', 'pressure', 'pressure_argoqc', 'salinity', 'salinity_argoqc', 'salinity_sfile', 'sali

Exception: (500, {'code': 500, 'message': 'Server error'})

Now let's indicate the data of interest within the collections we selected above. As examples, for each collection, we need to specify the variable name and the quality control flag of interest (if a specific qc flag is of interest and if the flag is available for a collection).

In [ ]:
#### params varying with the region

#+++ example to use Argo and cchdo profile data, as well as the glodap gridded product (which provides time mean fields)
selection_params['varnames']     = ['doxy', 'doxy', 'doxy', 'oxygen']
selection_params['varnames_qc']  = [',1',',2', '',''] #[',1', ''] # argoqc = 1 is best quality


#+++ example to use oxygen from Argo profile data (with argo qc flag 1) and the glodap gridded product
# selection_params['varnames']     = ['doxy', 'oxygen']
# selection_params['varnames_qc']  = [',1', ''] #[',1', ''] # argoqc = 1 is best quality
#+++

#+++ example to use salinity from Argo profile data (with argo qc flag 1) and the Roemmich and Gilson monthly gridded product
# selection_params['varnames']     = ['salinity', 'rg09_salinity']
# selection_params['varnames_qc']  = [',1', ''] # argoqc = 1 is best quality
#+++

# if we wanted to only consider oxygen profiles where temperature and salinity are qc 1, we would have here: 'temperature,1,salinity,1' (these additional variables are also stored for the regions of interest)
# (these additional variables are also stored for the regions of interest and visualized)
### please note that the first comma should be included!
### please note that qc flags are not used for Easyocean
selection_params['data_extra']     = ['','','',''] 

# define name of the variable that includes levels for each collection
selection_params['varname_levels'] = ['pressure','pressure','pressure',''] # for the gridded product, the level info is in the metadata, i.e. there is no variable in 'data' (for argo, 'pressure' is within the 'data' instead)


#### Before we continue, let's check that variables selected are included in the corresponding collection selected (if not, we should change the selected variable name in selection_params['varnames'])

In [ ]:
for i,ilist in enumerate(vars_lists):
    if selection_params['varnames'][i] not in ilist:
        print('>>>>> '+selection_params['collections'][i]+' does not include selected variable, i.e. '+selection_params['varnames'][i])
        print('Here is a list of the variables that are included:')
        print(ilist)
        change_name_of_selected_variable_before_continuing
    else:
        print('>>>>> '+selection_params['collections'][i]+' includes selected variable, i.e. '+selection_params['varnames'][i])

Let's now set the regions of interest and the type and tag for each region. Please note that in some regions and time periods of interest, the data of interest may not be available.


In [ ]:
#### params varying with the region

# in this example, we will use the 'box' selection type (i.e. we query the data within boxes),
# hence we indicate here the bottom/left and top/right vertices for each box of interest... 
# the other option is to search in a 'polygon' and indicate the polygon
# vertices in a list (first and last vertex should be the same)

#+++ example to use Argo and cchdo profile data, as well as the glodap gridded product (which provides time mean fields)
#### please note that shipped based profiles are sparser than Argo profiles, hence we need to select a box where we think there are data of interest in the period of interest
selection_params['regions']     = [
                        [[-55.5,35.5],[-45.5,40.5]],
                        ]
#+++ 

#+++ example to use oxygen from Argo profile data (with argo qc flag 1) and the glodap gridded product or the Roemmich and Gilson gridded product
# this is for the example Atlantic vs Pacific
# selection_params['regions']     = [
#                         [[-50,45],[-40,50]],
#                         [[-145.5,45.5],[-135.5,50.5]],
#                         ]
#+++

selection_params['regions_type'] = ['box', 'box']

selection_params['regions_tag'] = ['Atlantic','Pacific',]
####



Finally, let's set up the time periods of interest (there could be more than one, e.g. to compare different seasons or different years). We also define the vertical levels that we want to use to interpolate observed profiles and colors to use for the line plots.

In [ ]:
# list of startDate and endDates of interest (note: these will not be used for glodap as glodap only provides a time mean)
selection_params['startDate']    = ['2021-01-01T00:00:00Z']
selection_params['endDate']      = ['2021-12-31T00:00:00Z']

#### other params
# custom levels for vertically integrated profiles
selection_params['interp_levels']= list(range(10,2001))[0::20]

# levels for vertically integrated profiles if we want to use the vertical axis of one of the grids in Argovis
# selection_params['interp_levels']= 'rg09_temperature_200401_Total' #list(range(10,2001))[0::20]

# colors to use for the line plots
colors = list(mcolors.TABLEAU_COLORS.keys())

In [ ]:
# if the user requested to interpolate on the vertical axis on a grid, the code here redefines 'interp_levels'  based on that
if isinstance(selection_params['interp_levels'],str):
    
    meta = avh.query('grids/meta', options={"id": 'rg09_temperature_200401_Total'},apikey=API_KEY, apiroot=get_route(selection_params['interp_levels']),verbose=True)
    selection_params['interp_levels']=meta[0]['levels']
#selection_params['interp_levels']

## Loading the data of interest using the Argovis API

In the next code cell, parameters selected above are used to query the data of interest and format them in a way that allows to easily visualize the data. Depending on how many regions/time periods were selected and the extent of each region/time period of interest, the next code cell may take some time to complete.

In [ ]:
api_output_formatted_list = get_api_output_formatted_list_1var_for_regions_and_timeranges(selection_params=selection_params,API_KEY=API_KEY)


## Visualization: map of profile locations in each region and time period of interest

In [ ]:
# plot map of lons and lats for data that are not gridded
api_output_formatted_list_1var_plot_lons_lats_map(api_output_formatted_list,flag_map_for_only_one_var=False)      

## Visualization: plot profiles shown in the maps above

In [ ]:
# plot the profiles shown above
api_output_formatted_list_1var_plot_profiles(api_output_formatted_list)


## Visualization: example map for gridded products (if available) in each region of interest

In [ ]:
# plot map at one level and time
api_output_formatted_list_1var_plot_map(api_output_formatted_list,ilev=0,itime=0)  
               

## Visualization: for each product, plot the average vertical structure (i.e. horizontal and time average) in each region and time period of interest. 


In [ ]:
# plot horizontal and time average
api_output_formatted_list_1var_plot_horizontal_and_time_ave(api_output_formatted_list,colors)  


In [ ]:
# import sys 
# sys.getsizeof(api_output_formatted_list) 

# Extra section: visualizing profiles from Argo platform and/or WOCE line

In the following, we will use some of the plots above to visualize profiles from a specific Argo platform and/or a WOCE line. In doing so, we will focus only on the object of interest (e.g. profiles from a specific Argo platform or WOCE line) and **NOT** select by region or time.

## Setup: define parameters to query data of interest

Let's indicate the collections of interest (i.e. what datasets in Argovis you would like to use as they include the oceanic property of interest) and the parameter of interest. The parameter of interest should be present in all the collections to consider (e.g. 'platform' is there for the Argo collection, yet 'woceline' is not there for Argo). We also list 1. all the variables that are available for each collection, and 2. all the values that are possible for the selected parameter (so that we can use the variable name and value of interest for the selected parameter, in following code cells). **Please first run the code with the current settings to familiarize yourself with the notebook: after that, if needed, then change the settings and run the notebook again.**

In [ ]:
#+++

selection_params = {}

#+++ example for Argo platform
selection_params['collections']  = ['argo']
selection_params['parameter_name']= 'platform'
varname_temperature = 'temperature'
varname_salinity = 'salinity'
#+++

#+++ example for WOCE line from the Easyocean gridded product
selection_params['collections']  = ['easyocean']
selection_params['parameter_name']= 'woceline'
varname_temperature = 'ctd_temperature'
varname_salinity = 'ctd_salinity'
#+++ 

# let's print to screen some info based on the selected collections (to be able to select further parameters in the next cells)
show_variable_names_for_collections(collections_list=selection_params['collections'],API_KEY=API_KEY,verbose=True)

values_for_param = list_values_for_parameter_to_api_query(selection_params=selection_params,API_KEY=API_KEY,verbose=False)


Let's list values available for the param of interest of interest.

In [ ]:
for icollection in values_for_param.keys():
    print('---> ' + icollection)
    if len(values_for_param[icollection]['param_vals'])<100:
        print(values_for_param[icollection]['param_vals'])
    else:
        print('The list of param values is too long, will only show first 100 possible values')
        print(values_for_param[icollection]['param_vals'][0:100])


For each collection, indicate the parameter value of interest, the variable of interest, and, if needed, the 'section_start_date' of interest (this is needed e.g. for Easyocean). Please note that a variable may not be available for some parameter values (e.g. many Argo platforms do not measure bgc variables). The code in the following will return no profiles if the selected variable is not available for the parameter value of interest.

In [ ]:
#### parameters to set for each collection based on the info printed above

#+++ argo example
selection_params['parameter']     = ['4903274']
#+++ 

#+++  easyocean example
selection_params['parameter']     = ['P16'] #['A10']
#+++

# based on the list above, let's set the section_start_date for the 'woceline' of interest:
# please note that if you don't edit below (see # +++*** after the if statement) the code 
# will select the latest occupation in the dataset
if selection_params['parameter_name']=='woceline':
    selection_params['section_start_date'] = []
    for i,icollection in enumerate(selection_params['collections']):
        print('>----> Occupations for selected woceline ' +selection_params['parameter'][i] + ':')
        print(values_for_param[icollection]['param_vals_extra'][selection_params['parameter'][0]]['time_boundaris'])
        print('--+ Selected occupation start date')
        bfr_param_vals_extra = values_for_param[icollection]['param_vals_extra'][selection_params['parameter'][i]]
        selection_params['section_start_date'].append(bfr_param_vals_extra['time_boundaris'][-1][0])#['2011-09-28T00:00:00.000Z']
        print(selection_params['section_start_date'])
        
# +++*** uncomment here to select the section start date manually
# selection_params['section_start_date'] = '2005-01-10T00:00:00.000Z'

# define name of variable of interest for each collection

#+++ argo example
selection_params['varnames']     = ['doxy']
selection_params['varnames_qc']  = [',1'] # argoqc = 1 is best quality
#+++

#+++ easyocean example
selection_params['varnames']     = ['doxy']
selection_params['varnames_qc']  = [''] # woceqc = 2 is best quality
#+++

selection_params['varname_title']     = 'Oxygen, umol/kg'

# if we wanted to only consider oxygen profiles where temperature and salinity are qc 1, we would have here: 'temperature,1,salinity,1' (these additional variables are also stored for the regions of interest)
# (these additional variables are also stored for the regions of interest and visualized)
### please note that the first comma should be included!
### please note that qc flags are not used for Easyocean

#+++ argo example
selection_params['data_extra']   = [',temperature,1,salinity,1']
#+++

#+++ easyocean example
selection_params['data_extra']   = [',ctd_temperature,ctd_salinity'] # we included these as we want to store them
#+++

# define name of the variable that includes levels for each collection
selection_params['varname_levels'] = ['pressure'] # 


Finally, let's define the vertical levels that we want to use to interpolate observed profiles and colors to use for the line plots. 

In [ ]:
# levels for vertically integrated profiles
selection_params['interp_levels']= list(range(10,2001))[0::20]
# colors to use for the line plots
colors = list(mcolors.TABLEAU_COLORS.keys())

## Loading the data of interest using the Argovis API

In the next code cell, parameters selected above are used to query the data of interest and format them in a way that allows to easily visualize the data. Depending on the size of the data reuqested, the next code cell may take some time to complete.

In [ ]:
api_output_formatted_list = get_api_output_formatted_list_1var_for_parameter(selection_params=selection_params,API_KEY=API_KEY)


In [ ]:
# plot map of lons and lats for data that are not gridded
api_output_formatted_list_1var_plot_lons_lats_map(api_output_formatted_list)
# example of how to change the figure size
fig = plt.gcf() # gcf: get current figure
fig.set_size_inches(10,15)


Let's compute e.g. potential density, N_squared, absolute salinity, conservative temperature from in-situ temperature and salinity and add it to items in the list api_output_formatted_list.

In [ ]:
api_output_formatted_list = api_output_formatted_list_include_gsw_fields(list_fields_to_include=['absolute_salinity','conservative_temperature','potential_density','Nsquared'],
                                                        api_output_formatted_list0=api_output_formatted_list,
                                                        varname_temperature=varname_temperature,varname_salinity=varname_salinity)


In [ ]:
# let's plot all the profiles for each variable
api_output_formatted_list_1var_plot_profiles(api_output_formatted_list)

In [ ]:
# plot horizontal and time average
api_output_formatted_list_1var_plot_horizontal_and_time_ave(api_output_formatted_list,colors)  


Now, let's create a shaded plot showing all the profiles (in an order based on a coordinate of interest, e.g. longitude, latitude, timestamp) and all the levels.

In [ ]:
# select the variable of interest for the sorting:
# woceline: look at the map above and choose what makes most sense between longitude and latitude
# argo platform: 'timestamp' is generally a good choice; the cycle number would be good too, yet 
# it is not currently stored in api_output_formatted_list
var2use_for_sorting = 'latitude'

api_output_formatted_list_parameter_2d_plot(api_output_formatted_list,var2use_for_sorting=var2use_for_sorting)


In [ ]:
# other options:
# draw the shape of interest in the maps showing profiles?
# bin in time? 
# plot coloring by month or season